# Hello world


In [1]:
from IPython.display import display, HTML, Image

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn import metrics
from scipy.spatial import distance 

%matplotlib inline
#%qtconsole

In [2]:
class TemplateMatchClassifier(BaseEstimator, ClassifierMixin):
    
    """
    Parameters
    ----------
    distance_metric string, optional (default = 'euclidean')
    Other type of distance metrics: manhattan, chebyshev, and mahalanobis distance.

    Attributes
    ----------
    classes_ : array of shape = [n_classes] 
               The class labels (single output problem).
    templates_: dict
               A dictionary of the templates used for each class 
              {classes:templates}
    Notes
    -----


    See also
    --------
    
    
    ----------

    Examples
    --------
    >>> from sklearn.datasets import load_iris
    >>> from sklearn.model_selection import cross_val_score
    >>> clf = TemplateMatchClassifier()
    >>> iris = load_iris()
    >>> cross_val_score(clf, iris.data, iris.target, cv=10)
    
    """
    
    # Constructor for the classifier object
    def __init__(self, identifier = 'euclidean'):
        self.identifier = identifier
        
    # The fit function to train a classifier
    def fit(self, X, y):
        
        """Build a decision tree classifier from the training set (X, y).
        Parameters
        ----------
        X : array-like or sparse matrix, shape = [n_samples, n_features]
            The training input samples. Internally, it will be converted to
            ``dtype=np.float32`` and if a sparse matrix is provided
            to a sparse ``csc_matrix``.
        y : array-like, shape = [n_samples] 
            The target values (class labels) as integers or strings.
        Returns
        -------
        self : object
        """
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        
        # Count the number of occurrences of each class in the target vector (uses mupy unique function that returns a list of unique values and their counts)
        unique, counts = np.unique(y, return_counts = True)
        
        # Store the classes seen during fit
        self.classes_ = unique
        
        # # Obtain the distance matrix
        # if self.distance_metric == 'euclidean':
        #     self.distance_fn = distance.euclidean
        # elif self.distance_metric == 'cosine':
        #     self.distance_fn = distance.cosine
        # elif self.distance_metric == 'minkowski':
        #     self.distance_fn = distance.minkowski
        # elif self.distance_metric == 'manhattan':
        #     self.distance_fn = distance.cityblock
    
        # Set up a dictionary for each template
        self.templates_ = dict()
        
        # Calculate the mean of each template 
        for Class in self.classes_:
            self.templates_[Class] = X[y == Class].mean(axis = 0)
            
        return self
    
    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):
        
        """Predict class labels of the input samples X.
        Parameters
        ----------
        X : array-like or sparse matrix of shape = [n_samples, n_features]
            The input samples. Internally, it will be converted to
            ``dtype=np.float32`` and if a sparse matrix is provided
            to a sparse ``csr_matrix``.
        Returns
        -------
        p : array of shape = [n_samples, ].
            The predicted class labels of the input samples. 
        """
        
        # Check is fit had been called by confirming that the distributions_ dictionary has been set up
        check_is_fitted(self, ['templates_'])

        # Check that the input features match the type and shape of the training features
        X = check_array(X)

        # Initialise an empty list to store the predictions made
        predictions = list()
        
        